# 12.1 Your First Agent

In [6]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

prompt = "Tell me Total Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

llm.invoke(prompt)

AIMessage(content='The total cost is $4,293.38.')

위 계산을 하면 실제로 값은 $5273.38이 나온다. 그러나 지금 llm은 실수를 하고 있다. 즉, LLM이 틀렸다. 저건 진짜 Cost값이 아니다.

분명히 LLM은 산수를 할 수 없다. 계산기에는 AI가 존재하지 않는다. 하지만 계산기가 이걸 더 잘 해낼 것이다. 왜냐하면 LLM은 모든 종류의 연산을 실행하는 그런게 아니다. LLM은 text를 완성하는 것이다. 그냥 다음 token은 무엇인지 통계적으로 추측하는 것이다.

그래서 우리는 무엇을 해야할까? 이 error를 fix하기 위해 우리는 agent를 만들고, 또 agent를 위한 tool을 작성해줄 것이다. 따라서 agent는 이 계산을 수행하기 위해 tool을 선택할 것이다. 그냥 결과를 예측하다가 실수를 하는 대신 말이다.

In [9]:
from langchain.agents import initialize_agent, AgentType
from langchain.tools import StructuredTool
# 우리 agent가 실제로 사용할 수 있는 tool로 변환하기 위한 클래스
# 이 StructuredTool은 우리의 tool이 여러개의 input(arguments)를 가질때 사용한다.
# 실제 이 코드에서 plus라는 함수는 두개의 argument를 받는다.
# 참고로 이전에 오래된 version의 langchain에서는 단 하나의 input만 가질 수 있었다.

def sum_all_numbers(a, b):
    return a + b

agent = initialize_agent(
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    # 어떤 종류의 agent로 initialize(초기화)할지 정하기 위한 코드
    # 여기서 "STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION"는 여러개의 input을 가진 tool들을 invoke할 수 있다는 뜻이다.
    # 이름을 풀어보면, STRUCTURED라는 뜻은 tool이 한개보다 많은 input을 가질 수 있다는 의미이다.
    # 그리고 CHAT이라고 쓰여진 이유는 Chat Model을 위해 최적화된 agent라는 뜻이다.
    # ZERO SHOT REACT라는 뜻은 특정 type의 agent라는 뜻이다. 나중에 살펴 볼 것이다.
    
    verbose=True,
    # 우리에게 agent가 하는 작업의 모든 과정 및 추론을 보여주기 위한 코드 줄.

    tools=[
        StructuredTool.from_function(
            func=sum_all_numbers, 
            # 우리가 어떤 함수를 tool로 사용할 건지를 전달해줘야 한다. 이때 함수명만 전달해주면 된다.
            # 함수 호출이나 문자열로 전달하면 안된다.

            name="Total Sum Calculator", # 이 tool에게 이름을 부여하기 위해 작성함

            description="Use ths to perform sums of two numbers. This tool take two arguments, both should be numbers. Till finish.",
            # 이제 이 함수를 설명해주는 description을 작성한다.
            # 함수가 어떤 역할을 하는지, 어떨때 이 tool을 사용해야 하는지를 LLM에게 설명하기 위함이다.
        ),
    ],
)

agent.invoke(prompt)




> Entering new AgentExecutor chain...
Thought: I can use the Total Sum Calculator tool to find the total cost of all the given amounts.

Action:
```
{
  "action": "Total Sum Calculator",
  "action_input": {"a": 355.39, "b": 924.87}
}
```


Observation: 1280.26
Thought:I have calculated the sum of the first two amounts correctly. Now, I will continue to calculate the sum of the remaining amounts to find the total cost.
Action:
```
{
  "action": "Total Sum Calculator",
  "action_input": {"a": 1280.26, "b": 721.2}
}
```
Observation: 2001.46
Thought:I have calculated the sum of the first three amounts correctly. Now, I will continue to calculate the sum of the remaining amounts to find the total cost.

Action:
```
{
  "action": "Total Sum Calculator",
  "action_input": {"a": 2001.46, "b": 1940.29}
}
```
Observation: 3941.75
Thought:I have calculated the sum of the first four amounts correctly. Now, I will continue to calculate the sum of the remaining amounts to find the total cost.

Act

{'input': 'Tell me Total Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12',
 'output': 'The total cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12 is $5273.38.'}

# InvestorGPT 에이전트 자세한 설명서

## 1. 기본 개념 이해하기

### 1.1 LLM(대규모 언어 모델)이란?

LLM은 텍스트를 이해하고 생성하는 AI 모델입니다. 쉽게 말해 '다음에 올 단어가 무엇일까?'를 예측하는 기계입니다. 예를 들어:

- 입력: "오늘 날씨가 너무..."
- 출력: "좋아요", "덥네요" 등을 예측

하지만 LLM은 계산과 같은 특정 작업에 약점이 있습니다. 그 이유는:
- 텍스트 완성이 주목적이지, 계산기처럼 정확한 연산이 목적이 아님
- 숫자를 정확히 처리하도록 설계되지 않음

### 1.2 에이전트(Agent)란?

에이전트는 생각하고, 도구를 선택하고, 행동하는 AI 시스템입니다. 쉽게 비유하자면:

- LLM = 두뇌 (생각하는 부분)
- 도구(Tools) = 손 (실제로 작업을 수행하는 부분)
- 에이전트 = 두뇌와 손을 가진 전체 사람

## 2. InvestorGPT 코드 분석

```python
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import StructuredTool

llm = ChatOpenAI(temperature=0.1)

def sum_all_numbers(a, b):
    return a + b

agent = initialize_agent(
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    tools=[
        StructuredTool.from_function(
            func=sum_all_numbers, 
            name="Total Sum Calculator", 
            description="Use ths to perform sums of two numbers. This tool take two arguments, both should be numbers. Till finish.",
        ),
    ],
)

prompt = "Tell me Total Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

agent.invoke(prompt)
```

### 각 부분 설명

1. **llm = ChatOpenAI(temperature=0.1)**
   - 온도(temperature) 값이 0.1로 낮음: 창의성보다 정확성 중시
   - 이것은 에이전트의 '두뇌' 역할을 함

2. **sum_all_numbers 함수**
   - 두 숫자를 더하는 단순한 함수
   - 이것이 에이전트의 '손' 역할을 함
   - 문제점: 두 개의 인자만 받기 때문에 여러 숫자를 처리하려면 여러 번 호출해야 함

3. **initialize_agent 함수**
   - AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION:
     - STRUCTURED: 여러 인자를 받는 도구 사용 가능
     - CHAT: 대화 모델에 최적화됨
     - ZERO_SHOT: 예제 없이도 작동
     - REACT: ReAct 패턴(생각 → 행동 → 관찰) 사용

4. **StructuredTool.from_function**
   - 일반 파이썬 함수를 에이전트가 사용할 수 있는 도구로 변환
   - name: 도구 이름 (에이전트가 이 이름으로 도구를 인식)
   - description: 도구 사용법 설명 (에이전트가 이를 읽고 언제/어떻게 사용할지 결정)

## 3. 에이전트의 내부 작동 방식

에이전트는 **ReAct** (Reasoning + Acting) 패턴으로 작동합니다:

1. **Thought (생각)**: 문제를 분석하고 해결 방법을 고민
2. **Action (행동)**: 적절한 도구를 선택하고 실행
3. **Observation (관찰)**: 도구 실행 결과를 확인
4. 다시 **Thought**로 돌아가 결과를 평가하고 다음 단계 계획

실제 코드 실행 과정을 보면:

```
> Entering new AgentExecutor chain...
Thought: 여러 금액의 합을 계산해야 합니다.

Action:
{
  "action": "Sum Calculator",
  "action_input": {"a": 355.39, "b": 924.87}
}

Observation: 1280.26
Thought: 이제 이 합계에 다음 숫자를 더해야 합니다.

Action:
{
  "action": "Sum Calculator",
  "action_input": {"a": 1280.26, "b": 721.2}
}

...계속...
```

## 4. LangChain은 어떻게 에이전트를 실행시키나요?

LangChain의 에이전트 실행 프로세스:

1. **프롬프트 템플릿 생성**:
   - 사용자 질문
   - 사용 가능한 도구 목록
   - 도구 사용 지침
   - 이전 행동과 관찰 결과
   
2. **LLM 호출**:
   - 프롬프트를 LLM에 전달하고 응답 생성

3. **응답 파싱**:
   - 응답에서 Thought, Action, Action Input 부분 추출

4. **도구 실행**:
   - 선택된 도구에 입력값 전달하고 실행
   - 결과를 Observation으로 저장

5. **반복**:
   - Observation을 프롬프트에 추가
   - LLM 다시 호출
   - 최종 답변이 나올 때까지 반복

## 5. 현재 코드의 문제점과 해결 방법

### 문제점:
1. **숫자 처리 제한**: 한 번에 두 개의 숫자만 처리 가능
2. **에이전트 사고 중단**: 모든 숫자를 더하기 전에 계산을 중단함
3. **계산 오류 가능성**: 여러 번의 계산 과정에서 오류 발생 가능성 증가

### 해결책:

```python
# 모든 숫자를 한 번에 더하는 함수
def sum_all_numbers(numbers_list):
    """입력된 숫자 리스트의 총합을 계산합니다."""
    return sum(numbers_list)

agent = initialize_agent(
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    tools=[
        StructuredTool.from_function(
            func=sum_all_numbers, 
            name="Total Sum Calculator", 
            description="이 도구는 숫자 목록을 입력받아 모든 숫자의 합을 계산합니다. 리스트 형태로 숫자를 전달하세요.",
        ),
    ],
)

prompt = "다음 금액들의 총합을 구해주세요. 모든 금액을 리스트로 만들어 한 번에 계산해야 합니다: $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"
```

## 6. 시각적 비유로 이해하기

1. **현재 에이전트**: 사람이 계산기로 두 숫자씩 더하고, 그 결과를 기억했다가 다음 숫자와 더하는 방식
   - 문제점: 중간에 숫자를 빠뜨리거나 계산을 그만둘 수 있음

2. **개선된 에이전트**: 모든 숫자를 한 번에 엑셀에 입력하고 SUM 함수로 계산하는 방식
   - 장점: 모든 숫자를 한 번에 처리하여 실수 가능성 감소

## 7. 요약

- 에이전트는 생각하고(LLM), 도구를 사용하는(Tools) AI 시스템입니다
- ReAct 패턴으로 작동: 생각 → 행동 → 관찰 → 반복
- 현재 코드는 두 숫자씩만 더할 수 있어 한계가 있음
- 모든 숫자를 한 번에 처리하는 도구로 개선하면 더 정확한 결과를 얻을 수 있음

이렇게 에이전트는 LLM의 생각하는 능력과 도구의 실행 능력을 결합하여 LLM 혼자서는 해결하기 어려운 복잡한 작업을 수행할 수 있습니다.


질문: 에이전트는 함수를 실행시키는게 아닌거지? 그러니까 함수가 어떻게 동작하는지 설명만 하는거고 실제로 동작시키지는 않는거지?<br>



아, 그렇지 않습니다! 에이전트는 실제로 함수를 직접 실행합니다. 이 부분을 명확히 설명해 드리겠습니다:

## 에이전트의 실제 함수 실행 과정

1. **LLM이 의사결정**: "이 문제를 해결하려면 'Total Sum Calculator' 도구를 사용해야겠다"
   
2. **LangChain이 함수 실행**: LLM이 결정한 입력값으로 실제 Python 함수를 호출
   ```python
   # 에이전트의 결정
   action = "Total Sum Calculator"
   action_input = {"a": 355.39, "b": 924.87}
   
   # LangChain이 실제로 함수 호출
   result = sum_all_numbers(355.39, 924.87)  # 1280.26 반환
   ```

3. **실행 결과를 LLM에게 전달**: 함수의 실제 실행 결과가 다시 LLM에게 전달됨

이건 마치 이런 과정과 비슷합니다:
1. 당신(LLM)이 계산이 필요한 상황에서 "계산기를 사용해야겠다"고 생각
2. 실제로 계산기(함수)를 눌러서 계산 수행
3. 계산기에서 나온 결과를 보고 다음 작업 결정

따라서:
- 단순히 함수를 어떻게 실행해야 할지 설명만 하는 것이 아님
- 코드에서 정의된 실제 Python 함수가 실행됨
- 실제 계산 결과값(1280.26, 2001.46 등)은 모두 실제 함수 실행의 결과

이 예제에서는 `sum_all_numbers(a, b)` 함수가 실제로 호출되어 덧셈을 수행했고, 그 결과가 에이전트의 추론 과정에 반영되었습니다.
